## TSP problem

In [8]:
from CEM_optimizer.CEM import CEM
from CEM_optimizer.TSP import QAOA_TSP

In [1]:
import pennylane as qml


tsp = QAOA_TSP(
    number_of_cities=3,
    number_of_layers=6,
    optimization_steps=70,
    # optimizer=qml.GradientDescentOptimizer(stepsize=0.01)
    optimizer=qml.AdamOptimizer(
        stepsize=0.01,
        beta1=0.9,
        beta2=0.99
    )
)
cem = CEM(
    problem=tsp,
    epochs=10,
    samples_per_epoch=100,
    elite_frac=0.1,
    process=6
)

cem.minimize(init_weights=[1, 1, 1])

NameError: name 'QAOA_TSP' is not defined

## Knapsack problem

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from CEM_optimizer.Knapsack import Knapsack, QAOA_Knapsack
from CEM_optimizer.CEM import CEM

import pennylane as qml

In [10]:
knapsack = Knapsack(max_weight=2)
knapsack.set_knapsack([(2, 2), (3, 1)])

In [11]:
knapsack_qaoa = QAOA_Knapsack(
    knapsack, number_of_layers=2, 
    optimization_steps=200, 
    # optimizer=qml.AdagradOptimizer(stepsize=0.50)
    optimizer=qml.GradientDescentOptimizer(stepsize=0.01)
)

In [12]:
cem = CEM(
    problem=knapsack_qaoa,
    epochs=10,
    samples_per_epoch=100,
    elite_frac=0.1,
    process=6
)

cem.minimize(
    init_weights=[1, 1], 
    # mean=[2.0345243512510995, 2.942750372594455]
)

Epoch 1	Average Elite Score: -0.3782899219220698	Average Score: -0.14434978084974592
[0.40741604795471, 2.1311175186109237] with reward: -0.49233766576024335
Epoch 2	Average Elite Score: -0.40368561470809466	Average Score: -0.14922080741323795
[0.40741604795471, 2.1311175186109237] with reward: -0.49233766576024335
Epoch 3	Average Elite Score: -0.424884727586422	Average Score: -0.17815414332045523
[0.40741604795471, 2.1311175186109237] with reward: -0.49233766576024335
Epoch 4	Average Elite Score: -0.416870943769009	Average Score: -0.1918462343306206
[0.40741604795471, 2.1311175186109237] with reward: -0.49233766576024335
Epoch 5	Average Elite Score: -0.4572368882655226	Average Score: -0.22910839074397377
[0.40741604795471, 2.1311175186109237] with reward: -0.49233766576024335
Epoch 6	Average Elite Score: -0.4694387148076492	Average Score: -0.30410964102442567
[0.40741604795471, 2.1311175186109237] with reward: -0.49233766576024335


KeyboardInterrupt: 

In [13]:
cem.randomize(init_weights=[1, 1])

Epoch 1	Average Elite Score: -0.3825633806073557	Average Score: -0.12373419129605781
[-7.147861346391355, 9.347924610995836] with reward: -0.5678048497728833
Epoch 2	Average Elite Score: -0.362932978179451	Average Score: -0.11454479002779813
[6.689794925041902, -3.8811813854420674] with reward: -0.569855744441671
Epoch 3	Average Elite Score: -0.4507558604393235	Average Score: -0.14094499255572637
[6.689794925041902, -3.8811813854420674] with reward: -0.569855744441671
Epoch 4	Average Elite Score: -0.3481713818898989	Average Score: -0.12556784882292085
[6.689794925041902, -3.8811813854420674] with reward: -0.569855744441671
Epoch 5	Average Elite Score: -0.37607816600768434	Average Score: -0.12379451838521603
[6.689794925041902, -3.8811813854420674] with reward: -0.569855744441671
Epoch 6	Average Elite Score: -0.3731710878383884	Average Score: -0.1147461320326705
[6.689794925041902, -3.8811813854420674] with reward: -0.569855744441671
Epoch 7	Average Elite Score: -0.4510509117545601	Aver

[-4.575296603491294, -0.9036316684903625]

In [16]:
knapsack_qaoa.print_results([5.876428961941668, 1.9278045693719936])

Key: 101 with probability 0.5851258230403462   | correct: True, value: 2
Key: 111 with probability 0.23010160440914704   | correct: False
Key: 001 with probability 0.06187830273109165   | correct: False
Key: 011 with probability 0.05841408055146326   | correct: True, value: 1
Key: 100 with probability 0.03178243587187642   | correct: False
Key: 000 with probability 0.01852700194629529   | correct: True, value: 0
Key: 110 with probability 0.008241419311877003   | correct: False
Key: 010 with probability 0.005929332137900813   | correct: False


In [15]:
import pennylane as qml

def _create_mixing_hamitonian(const=1/2):
    hamiltonian = qml.Identity(0)
    for i in range(3):
        hamiltonian += qml.Hamiltonian([const], [qml.PauliX(i)])
    return hamiltonian
    

cost_operator = knapsack_qaoa._create_cost_operator((1, 1))
def qaoa_layer(gamma, beta):
    qml.qaoa.cost_layer(gamma, cost_operator)
    qml.qaoa.mixer_layer(beta, _create_mixing_hamitonian())

def hadamard_layer(wires):
    for i in range(wires):
        qml.Hadamard(i)

def circuit(params, n_layers=2):
    hadamard_layer(3)
    qml.layer(qaoa_layer, n_layers, params[0], params[1])

dev = qml.device("default.qubit", wires=3)


@qml.qnode(dev)
def cost_function(params):
    circuit(params, n_layers=1)
    x = qml.expval(cost_operator)
    return x

# fig, ax = qml.draw_mpl(cost_function, expansion_strategy="device")([[1], [2]])
# fig.show()
print(qml.draw(cost_function, expansion_strategy="device")([[1], [1]]))

0: ──H──MultiRZ(1.00)─╭MultiRZ(1.00)─╭MultiRZ(-1.00)──H───────────────MultiRZ(1.00)──H────────────
1: ──H────────────────╰MultiRZ(1.00)─│───────────────╭MultiRZ(-1.00)──H──────────────MultiRZ(1.00)
2: ──H──MultiRZ(2.00)────────────────╰MultiRZ(-1.00)─╰MultiRZ(-1.00)──H──────────────MultiRZ(1.00)

─────┤ ╭<𝓗>
───H─┤ ├<𝓗>
───H─┤ ╰<𝓗>


In [21]:
import numpy as np


elite_weights = [[8,2], [4,1], [2,0]]
print(np.var([8, 4, 2]))
_mean = np.mean(elite_weights, axis=0)
print(np.stack((elite_weights), axis = 1))
_cov = np.cov(np.stack((elite_weights), axis = 1), bias=True)
print(_mean)
print(_cov)

6.222222222222221
[[8 4 2]
 [2 1 0]]
[4.66666667 1.        ]
[[6.22222222 2.        ]
 [2.         0.66666667]]
